# Working with Data Classes - part 3

SO I had more issues with data classes, I found a few nice threads I am putting here for reference:

https://stackoverflow.com/questions/55099243/python3-dataclass-with-kwargsasterisk


https://towardsdatascience.com/12-examples-of-how-to-write-better-code-using-dataclass-d7c121ee0239

https://realpython.com/python-data-classes/


Now the question I had was this:

I know @dataclass is good to make code cleaner by avoiding repeating arguments that are placed as atributes.
So any argument or object we pass when instantiating a @dataclass becomes atributes?
or is it there another way? what about the case where you pass an argument to the Init that is used to instantiate a child object/atrib.
the answers in the first one kinda of related to this, for example when you instantiate an object with kwargs.
I guess there is need for more playtime with these issues.  did not find many good fits.
I guess one thing that comes to me  again and again is that  maybe the @dataclass is not the answer more often than it should.

In [10]:
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from typing import List

@dataclass
class Component(ABC):

    def __post_init__(self) -> None:
        return
        

    @property
    def parent(self) -> 'Component':
        return self._parent

    @parent.setter
    def parent(self, parent):
        self._parent = parent

  

@dataclass   
class Composite(Component):

    def __post_init__(self) -> None:
        super().__post_init__()
        self._children: List[Component] = []


    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self

        


In [12]:
composite_obj_one= Composite()
composite_obj_one._children

composite_obj_two=Composite()
composite_obj_two.add(composite_obj_one)
print(composite_obj_one.parent)

print(composite_obj_two._children)

Composite()
[Composite()]


In [16]:
from typing import Any

def from_float(x: Any) -> float:
    assert isinstance(x, (float, int)) and not isinstance(x, bool)
    return float(x)

@dataclass
class Settings(Composite):
    horizon_length: float
    interval_length: float
    
    
    
    def __post_init__(self):
        
        # self.horizon_length=horizon_length
        # self.interval_length=interval_length
        super().__post_init__()
        return

    @staticmethod
    def from_dict(obj: Any) -> 'Settings':
        assert isinstance(obj, dict)
        horizon_length = from_float(obj.get("horizon_length"))
        interval_length = from_float(obj.get("interval_length"))
        return Settings(horizon_length, interval_length)

@dataclass
class OptopyInput(Composite):
    settings: Settings
        
    def __post_init__(self):
        
        # self.settings=settings
        super().__post_init__()
    

    @staticmethod
    def from_dict(obj: Any) -> 'OptopyInput':
        assert isinstance(obj, dict)
        settings = Settings.from_dict(obj.get("settings"))
        
        return OptopyInput(settings)

In [18]:

optopy_inputs={
    "settings": {
      "horizon_length": 24.0,
      "interval_length": 0.25,
     
    },
}


data_obj = OptopyInput.from_dict(optopy_inputs)

now looks like if we do not place any arguments(atribs) in the base class, and add all the arguments in post init then the problem will be resolved.
But what if we wanted to have the parent class to be regular class and not data class, what if we wanted to have the inheriting class to be regular.